## Stable Diffusion Web UI Forge

**Homepage :** [SD Web UI Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Paperspace Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Paperspace-Guide)

## 1. Requirements

### 1.1. Initialization
**REQUIRED EVERY TIME YOU START THE MACHINE**

In [ ]:
import os
os.chdir('/')
from internal.main import *

initialization()

### 1.2. API Key
**Save your API Key in shared storage.**  
Some models hosted by CivitAI require you to login to download.  
Create an API key in [CivitAI user settings](https://civitai.com/user/account)

In [ ]:
# Run this cell then paste your API Key to the textfield
save_api_key()


### 1.3. Temporary Storage Settings (Optional)

Store files in the temporary storage. Settings will persist even if the machine is restarted.

In [ ]:
# When turning on/off the settings, all files in the associated directory will be deleted.
# So you need to download them again.

temp_storage_settings()

## 2. Web UI Extensions
Since the new version of Forge, almost extensions are broken.  
If you find any broken extensions, please report them to me on GitHub.

#### • Built-in Extensions
This is an information cell.  
To install/update these extensions, run the `Extension List` cell below.

* [Aspect Ratio and Resolution Buttons](https://github.com/thomasasfk/sd-webui-aspect-ratio-helper)
* [img2img Hires Fix](https://github.com/Amadeus-AI/img2img-hires-fix)
* [Infinite Image Browsing](https://github.com/zanllp/sd-webui-infinite-image-browsing)
* [SD Delete Button](https://github.com/reforget-id/sd_delete_button)
* [State](https://github.com/ilian6806/stable-diffusion-webui-state)
* [Ultimate SD Upscale](https://github.com/Coyote-A/ultimate-upscale-for-automatic1111)

#### • Extension List
Run the cell below to display the extension list

In [ ]:
extensions_selection(builtin_exts_forge_url, extensions_forge_url)
os.chdir(webui_path)


#### • Install from URLs

In [ ]:
# UPDATE EXTENSIONS
update_exts = False

# Install extensions from URL
other_exts = []


#########################################################################################################

install_other_exts(other_exts, update_exts)
os.chdir(webui_path)
completed_message()

## 3. Launch Web UI
**Before running this cell**, download models using `sd15_resource_lists.ipynb` or `sdxl_resource_lists.ipynb` or `flux_resource_lists.ipynb`.  
Run this cell and wait until `Startup time:  *.*s ...` appears.  
Then, click link `https://tensorboard-xxxxx.paperspacegradient.com` or ngrok link `https://xxxxx.ngrok-free.app` to open Web UI.  
Always watch the output log to track image generation progress or errors. 

In [ ]:
# Dark theme
webUI.dark_theme = True

# Authentication (Recommended for security purposes)
webUI.username = ''
webUI.password = ''

# Ngrok (Optional) 
# Some users reported that using ngrok can get you banned. So, this option will be disabled temporarily. 
# If you want to take the risk or have never had problems with ngrok before, you can remove the hashtags.
# webUI.ngrok_token = ''
# webUI.ngrok_domain = ''

# CORS (Optional) 
# separate with commas
webUI.cors = 'https://huchenlei.github.io'


#########################################################################################################
launch_webui(webUI)